# モデル作成

### ライブラリ,データのインポート

In [2]:
#ライブラリインポート

#import lightgbm as lgb
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#import plotly.plotly as py
#from plotly.graph_objs import*
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
# Pandasでの表示が省略されないように設定
pd.set_option("max_columns", 100)
pd.set_option('max_rows',100)
pd.set_option('max_info_columns',100)

#まじない
plt.style.use('ggplot') 



In [5]:
# データの読み込み（csvファイル）
# ヘッダーが無いcsvファイルを読み込むときは引数に'header=None'を指定
X = pd.read_csv('X.csv',index_col=0)
#X = df_train.drop(['y'], axis=1).values
#y = df_train['y'].values
test_X = pd.read_csv('test_X.csv',index_col=0)
#test_X = df_test.drop(['y'], axis=1).values
#sample = pd.read_csv('submit_sample.csv',header=None)

In [6]:
X.head()

,date,week,age_log_std,duration_log_std,duration_inv,campaign_log,balance_p_log_std,balance_m_log_std,balance_sign,pdays,pdays_log_std,previous_date,previous_log,total_touch,freq,education_num,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,poutcome_failure,poutcome_other,poutcome_success,contact_cellular,contact_telephone,contact_unknown,default,housing,loan,y
month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,93,2,-0.062289,1.816367,0.001064,0.000000,0.674389,-10.0,1,1000,5.000000,-1000.0,-1.000000,1,0.001000,2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1.0
2,49,0,0.991491,-0.020013,0.005780,2.302585,0.559045,-10.0,1,1000,5.000000,-1000.0,-1.000000,10,0.010000,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
4,103,5,-0.376709,1.269812,0.001761,0.000000,-0.144157,-10.0,1,595,1.549501,-492.0,0.693147,3,0.005042,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1.0
1,25,4,1.821543,0.952585,0.002358,0.000000,-0.095058,-10.0,1,1000,5.000000,-1000.0,-1.000000,1,0.001000,2,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
4,120,1,-0.964091,1.137957,0.001988,0.000000,-0.266570,-10.0,1,9,-4.050458,111.0,0.693147,3,0.333333,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1.0


In [7]:
test_X.head()

,date,week,age_log_std,duration_log_std,duration_inv,campaign_log,balance_p_log_std,balance_m_log_std,balance_sign,pdays,pdays_log_std,previous_date,previous_log,total_touch,freq,education_num,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,poutcome_failure,poutcome_other,poutcome_success,contact_cellular,contact_telephone,contact_unknown,default,housing,loan,y
month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,35,0,-1.092894,2.163971,0.000772,0.693147,0.359801,-10.000000,1,1000,5.0,-1000.0,-1.0,2,0.002,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
6,169,1,-0.062289,1.915569,0.000971,0.000000,-0.157789,-10.000000,1,1000,5.0,-1000.0,-1.0,1,0.001,3,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN
6,156,2,-0.164325,-2.035265,0.037037,3.178054,-10.000000,0.877531,-1,1000,5.0,-1000.0,-1.0,24,0.024,3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,NaN
5,147,0,-0.601235,-0.693435,0.010753,1.386294,-10.000000,0.745712,-1,1000,5.0,-1000.0,-1.0,4,0.004,2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,NaN
7,212,2,-0.269082,0.902844,0.002469,0.693147,-1.290177,-10.000000,1,1000,5.0,-1000.0,-1.0,2,0.002,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN


In [9]:
y = X['y']
#X = X.drop(['y','id'],axis=1)
X = X.drop(['y'],axis=1)
#test_X = test_X.drop('id',axis=1)
test_X = test_X.drop(['y'],axis=1)
#X = pd.get_dummies(X)
#test_X = pd.get_dummies(test_X)

## ランダムフォレスト

In [10]:
# データセットの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [11]:
# ①ランダムフォレストによる学習
from sklearn.ensemble import RandomForestClassifier
rfc_1 = RandomForestClassifier(random_state=0, n_estimators=2000)
rfc_1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [12]:
feat_labels = X.columns[1:]
importances = rfc_1.feature_importances_

indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

 1) duration_inv                   0.152259
 2) campaign_log                   0.151184
 3) week                           0.075763
 4) duration_log_std               0.067185
 5) balance_m_log_std              0.061966
 6) contact_cellular               0.039512
 7) age_log_std                    0.031671
 8) education_num                  0.031428
 9) previous_log                   0.029613
10) pdays_log_std                  0.024923
11) previous_date                  0.024325
12) freq                           0.021449
13) balance_p_log_std              0.019520
14) loan                           0.017259
15) total_touch                    0.012155
16) job_admin.                     0.011625
17) marital_single                 0.009655
18) month_11                       0.009180
19) education_tertiary             0.008722
20) default                        0.008597
21) month_7                        0.008481
22) education_primary              0.008340
23) job_retired                 

IndexError: index 53 is out of bounds for axis 0 with size 53

In [13]:
# 正答率を出力
print('train accuracy: %.3f' % rfc_1.score(X_train, y_train))
print('test  accuracy: %.3f' % rfc_1.score(X_test, y_test))

train accuracy: 1.000
test  accuracy: 0.908


In [14]:
pred1 = rfc_1.predict_proba(test_X)
pred1 = pred1[:,1]
pred1

array([5.91e-01, 6.34e-01, 1.50e-03, ..., 5.00e-04, 1.00e-03, 1.00e-03])

## Xgboost

In [16]:
#X2 = X.drop(['previous_dum','campaign_dum'],axis=1)
#test_X2 = test_X.drop(['previous_dum','campaign_dum'],axis=1)

In [14]:
# データセットの分割
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size=0.3, random_state=0)

In [17]:
#③Xgboostによる学習
# 必要なライブラリのインポート

# 動かすパラメータを明示的に表示
params = {"learning_rate":[0.08,0.1,0.12],
         "max_depth": [6,7],
         "subsample":[0.95,0.98],
         "colsample_bytree": [0.6,0.7],
         "min_child_weight":[3,3.5,3.8]
         }


# モデルにインスタンス生成
mod = xgb.XGBClassifier()
# ハイパーパラメータ探索
cv = GridSearchCV(mod, params, cv = 10, scoring= 'roc_auc', n_jobs =-1,verbose=10)

# 予測モデルを作成
cv.fit(X_train, y_train)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95 
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95, score=0.9277346476392959, total=   3.7s
[CV]  colsample_bytree=0.6, lear

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.9s


[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98 
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95, score=0.9296227642013184, total=   3.7s
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98 
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98, score=0.9384053302408486, total=   3.6s
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95, score=0.9089059595215546, total=   3.7s
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98 
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.95, score=0.9343284306979982, total=   3.6s
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_chil

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.9s


[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3.5, subsample=0.95 
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98, score=0.9317462501720105, total=   3.6s
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98, score=0.9332680368142854, total=   3.6s
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3.5, subsample=0.95 
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98, score=0.9299276611704267, total=   3.6s
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3.5, subsample=0.95 
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3.5, subsample=0.95 
[CV]  colsample_bytree=0.6, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.98, score=0.9083906026976637, total=   3.6s
[CV] colsample_bytree=0.6, learning_rate=0.08, max_depth=6, 

...

[Parallel(n_jobs=-1)]: Done 669 tasks      | elapsed:  6.7min


[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3, subsample=0.98, score=0.9386428577210435, total=   5.4s
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.5, subsample=0.95 
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3, subsample=0.98, score=0.933945285834163, total=   5.4s
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.5, subsample=0.95 
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3, subsample=0.98, score=0.9363385921314996, total=   5.4s
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.5, subsample=0.95 
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3, subsample=0.98, score=0.9351405898811981, total=   5.4s
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.5, subsample=0.95 
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, 

[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.98 
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.98 
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.95, score=0.9087737475614989, total=   5.4s
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.98, score=0.9383543257202375, total=   5.3s
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.98 
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.98, score=0.9268219351652008, total=   5.3s
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.98 
[CV] colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=0.98 
[CV]  colsample_bytree=0.7, learning_rate=0.12, max_depth=7, min_child_weight=3.8, subsample=

[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  7.2min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'learning_rate': [0.08, 0.1, 0.12], 'max_depth': [6, 7], 'subsample': [0.95, 0.98], 'colsample_bytree': [0.6, 0.7], 'min_child_weight': [3, 3.5, 3.8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [18]:
cv.best_params_

{'colsample_bytree': 0.6,
 'learning_rate': 0.12,
 'max_depth': 6,
 'min_child_weight': 3.5,
 'subsample': 0.95}

In [19]:
pred2 = cv.predict_proba(test_X)
pred2 = pred2[:,1]
pred2

array([6.2231696e-01, 6.5219015e-01, 6.6271570e-04, ..., 4.4707328e-04,
       3.5979999e-03, 1.0634545e-02], dtype=float32)

## LightGBM

In [20]:
#④LightGBMによる学習
# LightGBM parameters
params2 = {
        'max_depth': [4,5,6,7,8,9],
        'learning_rate': [0.08,0.09,0.1,0.11,0.12,0.13]
}


# モデルにインスタンス生成
gbm = lgb.LGBMClassifier()
# ハイパーパラメータ探索
gbm_cv = GridSearchCV(gbm, params2, scoring= 'roc_auc', n_jobs =-1,verbose=10)

# 予測モデルを作成
gbm_cv.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] learning_rate=0.08, max_depth=4 .................................
[CV] learning_rate=0.08, max_depth=4 .................................
[CV] learning_rate=0.08, max_depth=4 .................................
[CV] learning_rate=0.08, max_depth=5 .................................
[CV] learning_rate=0.08, max_depth=5 .................................
[CV] learning_rate=0.08, max_depth=5 .................................
[CV] learning_rate=0.08, max_depth=6 .................................
[CV] learning_rate=0.08, max_depth=6 .................................
[CV]  learning_rate=0.08, max_depth=4, score=0.9162784629462866, total=   1.4s
[CV]  learning_rate=0.08, max_depth=4, score=0.9270841058437413, total=   1.4s
[CV]  learning_rate=0.08, max_depth=4, score=0.9314798181900695, total=   1.4s
[CV] learning_rate=0.08, max_depth=6 .................................
[CV] learning_rate=0.08, max_depth=7 .........................

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.6s


[CV]  learning_rate=0.08, max_depth=6, score=0.9331157580983394, total=   1.9s
[CV]  learning_rate=0.08, max_depth=6, score=0.9300985323392851, total=   2.0s
[CV]  learning_rate=0.08, max_depth=5, score=0.9188177667525117, total=   2.1s
[CV] learning_rate=0.08, max_depth=7 .................................
[CV] learning_rate=0.08, max_depth=8 .................................
[CV] learning_rate=0.08, max_depth=8 .................................
[CV]  learning_rate=0.08, max_depth=5, score=0.9326673974722411, total=   2.2s
[CV]  learning_rate=0.08, max_depth=5, score=0.9297494182239265, total=   2.3s
[CV] learning_rate=0.08, max_depth=8 .................................
[CV] learning_rate=0.08, max_depth=9 .................................
[CV]  learning_rate=0.08, max_depth=6, score=0.9195929200962176, total=   2.3s
[CV]  learning_rate=0.08, max_depth=7, score=0.9338156617766439, total=   2.3s
[CV]  learning_rate=0.08, max_depth=7, score=0.9305685822016511, total=   2.3s
[CV] learning

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.1s


[CV]  learning_rate=0.08, max_depth=7, score=0.9206472111713349, total=   2.3s
[CV] learning_rate=0.09, max_depth=4 .................................
[CV]  learning_rate=0.08, max_depth=8, score=0.9325719009058705, total=   2.4s
[CV] learning_rate=0.09, max_depth=4 .................................
[CV]  learning_rate=0.08, max_depth=8, score=0.931529766916342, total=   2.5s
[CV]  learning_rate=0.08, max_depth=8, score=0.9213173844551839, total=   2.5s
[CV] learning_rate=0.09, max_depth=5 .................................
[CV] learning_rate=0.09, max_depth=5 .................................
[CV]  learning_rate=0.08, max_depth=9, score=0.9308683768290017, total=   2.6s
[CV] learning_rate=0.09, max_depth=5 .................................


...

[CV]  learning_rate=0.12, max_depth=7, score=0.930670220870089, total=   2.6s
[CV] learning_rate=0.13, max_depth=4 .................................
[CV]  learning_rate=0.12, max_depth=7, score=0.9216470097067112, total=   2.6s
[CV] learning_rate=0.13, max_depth=4 .................................
[CV]  learning_rate=0.12, max_depth=8, score=0.929547777860179, total=   2.6s
[CV] learning_rate=0.13, max_depth=4 .................................
[CV]  learning_rate=0.12, max_depth=8, score=0.9313041142153495, total=   2.7s
[CV] learning_rate=0.13, max_depth=5 .................................
[CV]  learning_rate=0.12, max_depth=8, score=0.9215931239881257, total=   2.8s
[CV] learning_rate=0.13, max_depth=5 .................................
[CV]  learning_rate=0.12, max_depth=9, score=0.9295225728147105, total=   2.8s
[CV] learning_rate=0.13, max_depth=5 .................................
[CV]  learning_rate=0.13, max_depth=4, score=0.9297864987235098, total=   1.8s
[CV] learning_rate=0.13

[Parallel(n_jobs=-1)]: Done 104 out of 108 | elapsed:   31.7s remaining:    1.2s


[CV]  learning_rate=0.13, max_depth=9, score=0.9323765394422661, total=   2.0s
[CV]  learning_rate=0.13, max_depth=9, score=0.9302115915095835, total=   2.2s
[CV]  learning_rate=0.13, max_depth=9, score=0.9230213382591029, total=   2.0s


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   32.2s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [4, 5, 6, 7, 8, 9], 'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12, 0.13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [21]:
gbm_cv.best_params_

{'learning_rate': 0.13, 'max_depth': 9}

In [22]:
pred3 = gbm_cv.predict_proba(test_X)
pred3 = pred3[:,1]
pred3

array([4.68912202e-01, 5.92887739e-01, 1.36395965e-04, ...,
       5.60392474e-05, 1.69354684e-03, 1.18967414e-02])

In [23]:
'''
pred2=pd.read_csv('XGBOOST1.csv',header=None)
pred2 = np.array(pred2.values)
pred2 = pred2[:,1]
pred2
'''

"\npred2=pd.read_csv('XGBOOST1.csv',header=None)\npred2 = np.array(pred2.values)\npred2 = pred2[:,1]\npred2\n"

In [24]:
print(pred1.shape)
print(pred2.shape) 
print(pred3.shape)

(18083,)
(18083,)
(18083,)


In [25]:
#Randomforest:Xgboost:LightGBM=0.9364:0.9361:0.9361=weights[0.4,0.3,0.3]★MAX（%）★
p = np.average(np.array([pred1,pred2,pred3]),axis=0,weights=[0.4,0.3,0.3])
sample[1] = p
sample[1].head()

0    0.558769
1    0.617123
2    0.001440
3    0.000417
4    0.150330
Name: 1, dtype: float64

In [26]:
sample.to_csv('ENSAMBLING2(RANDOMFOREST_XGBOOST_LIGHTGBM)_bank.csv',index=None,header=None)